<a href="https://colab.research.google.com/github/Matheusups/data-platform-colab/blob/main/core/functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import Libraries**

In [34]:
pip install s3fs

In [39]:
import s3fs
import pandas as pd

# **Funções**



### **listar_s3**
Responsável por listar arquivos e pastas dentro do bucket. Retorno um dataframe estruturado por **Caminho, tipo e objeto**.

In [40]:
def listar_s3(bucket_name,num_reg,tipo):
  """
  Args:
        bucket_nome (str): O nome do bucket S3.
        num_reg (str): Quantidade de registros desejados no retorno.
        tipo (str): Arquivo ou Pasta.

  Returns:
        pandas.DataFrame: Recebe a lista de pastas e arquivos dentro do bucket para que possa ser realizado a analise dos caminhos.
  """

  # Conectar ao S3
  fs = s3fs.S3FileSystem(anon=True)  # Usando anon=True para acesso público

  # Walk para percorrer storage acessado
  # Listagem de arquivos e pastas separadamente
  dados = []
  for dirpath, dirnames, filenames in fs.walk(bucket_name):
      for subpasta in dirnames:
          dados.append({"Caminho": dirpath, "Tipo": "Pasta", "Objeto": subpasta})
      for arquivo in filenames:
          dados.append({"Caminho": dirpath, "Tipo": "Arquivo", "Objeto": arquivo})

  df = pd.DataFrame(dados)

  if tipo is None:
    df_final = df.head(num_reg)
  elif tipo == 'Pasta':
    df_final = df[df['Tipo'] == tipo].head(num_reg)
  elif tipo == 'Arquivo':
    df_final = df[df['Tipo'] == tipo].head(num_reg)

  return df_final


In [53]:
df_list = listar_s3('s3://ons-aws-prod-opendata/',10000,'Arquivo')

### **download_list_s3**
Permite realizar o download dos dados no filesystem local de depuração da plataforma, assim facilitando a analise no formato desejado (Excel ou CSV).

In [58]:
def download_list_s3(dataframe,formato):
  """
  Args:
        dataframe (pandas.dataframe): Dataframe com informações de pastas e arquivos recebidos na função *listar_s3*
        formato (str): formato CSV ou Excel

  Returns:
        Arquivo no formato solicitado dentro do ./diretorio_atual
  """
  if formato == 'Excel':
    return dataframe.to_excel('./list_folders_files.xlsx', index=False)
  elif formato == 'CSV':
    return dataframe.to_csv('./list_folders_files.csv', index=False)

In [57]:
download_list_s3(df_list,'Excel')